# Technical Assessment - Analyst, Data Science

A Supervision Manager has asked you to help in allocating scarce resources, and identify which firms their team should focus on. Supervisory resource may be allocated according to the following characteristics: 
* Firm size (i.e. the biggest firms need more attention) 
* Changing business profile (are firms’ data changing substantially year-on-year?) 
* Outliers from the norm (when looking at a single reporting period, does a firm deviate significantly from the average?) 

Some typical metrics have been provided in the attached data sheets. These include: 
* Gross Written Premium (GWP) – total revenue written by an insurer. Equivalent of turnover for a non-insurance firm. 
* Net Written Premium (NWP) – GWP less reinsurance. NWP / GWP will show how much of the firm’s risk is being passed on to reinsurers. 
* SCR coverage ratio – a measure of whether a firm is meeting its prudential capital requirements. Greater than 100% means the firm is holding enough capital to meet the requirement. The size of the buffer (i.e. surplus over 100%) can be important. 
* Gross claims incurred – a large cost to an insurer. Monitoring how these change over time for a firm is vital. 
* Net combined ratio – (incurred losses plus expenses) / earned premiums. This is a ratio that can indicate the profitability of a firm. If this is less than 100% it indicates a profit. 

### Task 1

Using the data provided, please analyse this data using a programming language of your choosing and produce a short report, including tables and charts, to highlight which firms should receive the most attention, according to the metrics above. 

### Task 2

Please consider using relevant ML techniques to draw out further insights and present them as an annex to your report.

## 1. Read data

In [1]:
import pandas as pd
import os

os.chdir("../")

In [2]:
general_data = pd.read_excel('technical-assessment-data.xlsx', sheet_name = 'Dataset 1 - General')
underwriting_data = pd.read_excel('technical-assessment-data.xlsx', sheet_name = 'Dataset 2 - Underwriting')

## 2. Clean and wrangle

In [3]:
# Refine variables
general_data = general_data[['Unnamed: 0',
 'NWP (£m) ', 'NWP (£m) .1', 'NWP (£m) .2', 'NWP (£m) .3', 'NWP (£m) .4',
 'GWP (£m)', 'GWP (£m).1', 'GWP (£m).2', 'GWP (£m).3', 'GWP (£m).4',
 'SCR coverage ratio', 'SCR coverage ratio.1', 'SCR coverage ratio.2', 'SCR coverage ratio.3', 'SCR coverage ratio.4'
]]
underwriting_data = underwriting_data[['Unnamed: 0',
 'Gross claims incurred (£m)', 'Gross claims incurred (£m).1', 'Gross claims incurred (£m).2', 'Gross claims incurred (£m).3', 'Gross claims incurred (£m).4',
 'Net combined ratio', 'Net combined ratio.1', 'Net combined ratio.2', 'Net combined ratio.3', 'Net combined ratio.4'
]]

# Convert df from wide to long
general_rename_dict = {'Unnamed: 0': 'firm',
 'NWP (£m) ' : 'net-written-premium2016',
 'NWP (£m) .1' : 'net-written-premium2017',
 'NWP (£m) .2' : 'net-written-premium2018',
 'NWP (£m) .3' : 'net-written-premium2019',
 'NWP (£m) .4' : 'net-written-premium2020',
 'SCR coverage ratio' : 'scr-coverage-ratio2016',
 'SCR coverage ratio.1' : 'scr-coverage-ratio2017',
 'SCR coverage ratio.2' : 'scr-coverage-ratio2018',
 'SCR coverage ratio.3' : 'scr-coverage-ratio2019',
 'SCR coverage ratio.4' : 'scr-coverage-ratio2020',
 'GWP (£m)' : 'gross-written-premium2016',
 'GWP (£m).1' : 'gross-written-premium2017',
 'GWP (£m).2' : 'gross-written-premium2018',
 'GWP (£m).3' : 'gross-written-premium2019',
 'GWP (£m).4' : 'gross-written-premium2020'
}
general_data.rename(columns = general_rename_dict, inplace = True)
general_data.drop([0], inplace = True)
general_data = pd.wide_to_long(general_data, ['net-written-premium','scr-coverage-ratio', 'gross-written-premium'], i = 'firm', j = 'year')
general_data = general_data.reset_index(level=['firm', 'year'])

underwriting_rename_dict = {'Unnamed: 0' : 'firm',
 'Gross claims incurred (£m)' : 'gross-claims-incurred2016',
 'Gross claims incurred (£m).1' : 'gross-claims-incurred2017',
 'Gross claims incurred (£m).2' : 'gross-claims-incurred2018',
 'Gross claims incurred (£m).3' : 'gross-claims-incurred2019',
 'Gross claims incurred (£m).4' : 'gross-claims-incurred2020',
 'Net combined ratio' : 'net-combined-ratio2016',
 'Net combined ratio.1' : 'net-combined-ratio2017',
 'Net combined ratio.2' : 'net-combined-ratio2018',
 'Net combined ratio.3' : 'net-combined-ratio2019',
 'Net combined ratio.4' : 'net-combined-ratio2020'
}
underwriting_data.rename(columns = underwriting_rename_dict, inplace = True)
underwriting_data.drop([0], inplace = True)
underwriting_data = pd.wide_to_long(underwriting_data, ['gross-claims-incurred', 'net-combined-ratio'], i = 'firm', j = 'year')
underwriting_data = underwriting_data.reset_index(level=['firm', 'year'])

# Outer join into a single df
firm_data = pd.merge(general_data, underwriting_data, on = ['firm', 'year'], how = 'outer')

In [4]:
firm_data.head()

,firm,year,net-written-premium,scr-coverage-ratio,gross-written-premium,gross-claims-incurred,net-combined-ratio
0,Firm 1,2016,-17754.100486,0.16252,46.76803,0,0
1,Firm 2,2016,9.244369,5.648647,41.794375,44.467865,1.276296
2,Firm 3,2016,0,1.135498,0,0,0
3,Firm 4,2016,19677.453353,0.679158,43375.80696,14.340938,0.672481
4,Firm 5,2016,128.812208,1.463499,80.183916,48.738606,0.509871
